<a href="https://colab.research.google.com/github/armandossrecife/teste/blob/main/concatena_imagens_vertical_gera_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pillow

In [2]:
"""
images_to_vertical_pdf.py

Dado uma lista de imagens, empilha todas verticalmente em UMA ÚNICA página
e salva como PDF.

Requisitos: Pillow
    pip install pillow

Exemplos:
    python images_to_vertical_pdf.py img1.jpg img2.png img3.jpeg -o saida.pdf
    python images_to_vertical_pdf.py ./imagens/*.jpg -o juntou.pdf -w 1200 -m 8 --bg "#f8f9fa"
"""
from __future__ import annotations
import argparse
import sys
from pathlib import Path
from typing import List, Optional, Tuple

from PIL import Image, ImageOps


def load_and_prepare(path: Path, resize_width: Optional[int]) -> Image.Image:
    """Abre a imagem, respeita orientação EXIF, converte para RGB e redimensiona opcionalmente."""
    img = Image.open(path)
    # Respeitar orientação EXIF (caso vindo de celular/câmera)
    img = ImageOps.exif_transpose(img)
    # Converter para RGB (PDF não suporta transparência/alpha)
    if img.mode != "RGB":
        img = img.convert("RGB")
    # Redimensionar mantendo aspecto, se solicitado
    if resize_width is not None and img.width != resize_width:
        ratio = resize_width / float(img.width)
        new_h = max(1, int(img.height * ratio))
        img = img.resize((resize_width, new_h), Image.LANCZOS)
    return img


def images_to_vertical_pdf(
    image_paths: List[Path],
    output_pdf: Path,
    resize_width: Optional[int] = None,
    margin: int = 0,
    bg_color: str = "white",
    dpi: int = 300,
) -> None:
    if not image_paths:
        raise ValueError("Nenhuma imagem fornecida.")
    # Carregar e padronizar
    prepared: List[Image.Image] = [load_and_prepare(p, resize_width) for p in image_paths]
    widths = [im.width for im in prepared]
    heights = [im.height for im in prepared]

    max_w = max(widths)
    total_h = sum(heights)
    # Margem: entre imagens (margin px) + bordas (margin px em cima/baixo e esquerda/direita)
    # Vamos usar a mesma margem para borda e espaçamento entre imagens.
    if margin < 0:
        margin = 0
    total_h_with_margins = total_h + margin * (len(prepared) - 1) + 2 * margin
    total_w_with_margins = max_w + 2 * margin

    # Criar "lona" final (uma única página)
    canvas = Image.new("RGB", (total_w_with_margins, total_h_with_margins), color=bg_color)

    y = margin
    for im in prepared:
        # Centralizar horizontalmente cada imagem
        x = margin + (max_w - im.width) // 2
        canvas.paste(im, (x, y))
        y += im.height + margin

    # Salvar como PDF em uma única página
    # A Pillow converte a imagem única para 1 página de PDF.
    canvas.save(output_pdf, "PDF", resolution=dpi)


def parse_args(argv: List[str]) -> argparse.Namespace:
    parser = argparse.ArgumentParser(
        description="Empilha imagens verticalmente em uma única página PDF."
    )
    parser.add_argument(
        "imagens",
        nargs="+",
        help="Caminhos das imagens (suporta curingas do shell, ex.: ./imagens/*.jpg).",
    )
    parser.add_argument(
        "-o",
        "--output",
        type=str,
        required=True,
        help="Caminho do arquivo PDF de saída (ex.: saida.pdf).",
    )
    parser.add_argument(
        "-w",
        "--width",
        type=int,
        default=None,
        help="Largura final (em pixels) para padronizar todas as imagens mantendo o aspecto. "
             "Se omitido, mantém tamanhos originais.",
    )
    parser.add_argument(
        "-m",
        "--margin",
        type=int,
        default=0,
        help="Margem (em pixels) a usar nas bordas e entre imagens. Padrão: 0.",
    )
    parser.add_argument(
        "--bg",
        "--background",
        dest="bg",
        type=str,
        default="white",
        help='Cor de fundo (ex.: "white", "#ffffff"). Padrão: white.',
    )
    parser.add_argument(
        "--dpi",
        type=int,
        default=300,
        help="DPI usado ao salvar o PDF (ajuda em impressão). Padrão: 300.",
    )
    return parser.parse_args(argv)


def main(argv: List[str]) -> int:
    args = parse_args(argv)
    # Expandir e validar caminhos
    image_paths: List[Path] = []
    for p in args.imagens:
        # O shell normalmente expande curingas; se não, tratamos literalmente
        for path in sorted(Path().glob(p)) if any(ch in p for ch in "*?[]") else [Path(p)]:
            if not path.exists():
                print(f"Aviso: {path} não existe; ignorando.", file=sys.stderr)
                continue
            if path.is_file():
                image_paths.append(path)

    if not image_paths:
        print("Nenhuma imagem válida encontrada.", file=sys.stderr)
        return 1

    output_pdf = Path(args.output)
    try:
        images_to_vertical_pdf(
            image_paths=image_paths,
            output_pdf=output_pdf,
            resize_width=args.width,
            margin=args.margin,
            bg_color=args.bg,
            dpi=args.dpi,
        )
        print(f"PDF gerado em: {output_pdf.resolve()}")
        return 0
    except Exception as e:
        print(f"Erro: {e}", file=sys.stderr)
        return 2